In [169]:
import pandas as pd


In [170]:
df_main = pd.read_csv("heart.csv")
df_main.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [171]:
df_main.head(5)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [172]:
df_numeric = df_main[['Age','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak']].copy()
df_non_numeric = df_main[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']].copy()
df_target = df_main['HeartDisease']

In [173]:
import numpy as np

In [174]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)
df_scaled[~((df_scaled >= -3) & (df_scaled <= 3))] = np.nan

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputed = imputer.fit_transform(df_scaled)
df_scaled = pd.DataFrame(imputed,columns= df_numeric.columns)
df_scaled.max()

Age            2.491558
RestingBP      2.842807
Cholesterol    2.919751
FastingBS      1.813758
MaxHR          2.561874
Oldpeak        2.919951
dtype: float64

In [175]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
df_encoded = encoder.fit_transform(df_non_numeric)
df_encoded = df_encoded.toarray()
columns = encoder.get_feature_names_out()
final_columns = [name.split('_')[1] for name in columns]
df_encoded = pd.DataFrame(df_encoded,columns = final_columns)
df_encoded.head(5)

,F,M,ASY,ATA,NAP,TA,LVH,Normal,ST,N,Y,Slope,Slope,Slope
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [176]:
df_final = pd.concat([df_scaled,df_encoded],axis=1)
df_final.head(3)

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,F,M,ASY,ATA,NAP,TA,LVH,Normal,ST,N,Y,Slope,Slope,Slope
0,-1.433140,0.410909,0.825070,-0.551341,1.382928,-0.832432,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,-0.478484,1.491752,-0.171961,-0.551341,0.754157,0.105664,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-1.751359,-0.129513,0.770188,-0.551341,-1.525138,-0.832432,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [177]:
from sklearn.svm import SVC

In [178]:
from sklearn.model_selection import train_test_split,cross_val_score


In [179]:
X_train,X_test,y_train,y_test = train_test_split(df_final,df_target,test_size = 0.2, random_state = 1 )

In [180]:
model_svc = SVC()
model_svc.fit(X_train,y_train)

SVC()

In [181]:
model_svc.score(X_test,y_test)

0.8967391304347826

In [182]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(
    estimator = SVC(),
    n_estimators=100,
    max_samples = 0.8,
    oob_score = True, # Czyli bierze nie uzyte wcześniej dane, i na ich podstawie wylicza wynik ( score )
    random_state = 1
)
bagging.fit(X_train,y_train)
bagging.oob_score_

0.8651226158038147

In [183]:
bagging.score(X_test,y_test)

0.8967391304347826

In [184]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)
tree.score(X_test,y_test)

0.7880434782608695

In [185]:
bagging_tree = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators=100,
    max_samples = 0.8,
    oob_score = True, # Czyli bierze nie uzyte wcześniej dane, i na ich podstawie wylicza wynik ( score )
    random_state = 1
)
bagging_tree.fit(X_train,y_train)
bagging_tree.oob_score_

0.8433242506811989

In [186]:
bagging_tree.score(X_test,y_test)

0.8641304347826086